In [1]:
from sqlalchemy import extract, desc
import datetime
import time
import MetaTrader5 as mt5
import pandas as pd

import options_estrategies as opt_est
from options_estrategies import Option_Due, InformationType
import models as model

In [2]:
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

estrategies = opt_est.options_estrategies(model, model.PETR4, model.PETR4_OPTIONS)

In [3]:
def thl_convert_dict_from_update_ticks_to_dataframe(tick_data):
    df_tick_data = pd.DataFrame.from_dict(tick_data)
    df_tick_data = df_tick_data.drop(columns=['_sa_instance_state','timestamp_option','days_to_due_date'])
    df_tick_data['stock_price'] = df_tick_data.stock_price.astype(float)
    df_tick_data = df_tick_data[['option_name','deal_type_zone','stock_price','strike','bid','ask']]
    
    return df_tick_data


In [4]:
def thl_convert_dict_from_update_ticks_to_dataframe(data_thl):
    df_thl_compare = data_thl.copy()
    df_thl_compare["thl_cost"] = (df_thl_compare['bid_x'] - df_thl_compare['ask_y'])*-1
    df_thl_compare["thl_percent_of_strike"] = (df_thl_compare["thl_cost"] / df_thl_compare.index)*100
    df_thl_compare = df_thl_compare[['option_name_x','option_name_y','thl_percent_of_strike','thl_cost','bid_x','ask_y','deal_type_zone_y']]
    df_thl_compare = df_thl_compare[df_thl_compare['ask_y'] > 0]
    return df_thl_compare

In [8]:
returned_thl_operations = estrategies.thl_operation(mode=InformationType.Real_Time)

comparison_this_month = returned_thl_operations['thl']
comparison_next_month = returned_thl_operations['thl_next_month']
comparison_calendar = returned_thl_operations['thl_calendar']

good_operation_THL = comparison_this_month[comparison_this_month['thl_percent_of_strike'] < 1.5]
good_operation_THL_next_month = comparison_next_month[comparison_next_month['thl_percent_of_strike'] < 1.5]
good_operation_calendar = comparison_calendar[comparison_calendar['thl_percent_of_strike'] < 1.5]


good_operation_THL

,option_name_x,option_name_y,thl_percent_of_strike,thl_cost,bid_x,ask_y,deal_type_zone_y
strike,,,,,,,
28.96,PETRH243,PETRI289,1.484807,0.43,0.48,0.91,OTM


In [9]:
good_operation_THL_next_month


,option_name_x,option_name_y,thl_percent_of_strike,thl_cost,bid_x,ask_y,deal_type_zone_y
strike,,,,,,,


In [10]:
good_operation_calendar  

,option_name_x,option_name_y,thl_percent_of_strike,thl_cost,bid_x,ask_y,deal_type_zone_y
strike,,,,,,,


In [130]:
last_warnings = []
alarme_to_remove = []

def check_exists_alarm(operation, option_name, time_expiration = 4):
    for i in range(0,len(last_warnings)):
        if last_warnings[i]['timestamp'] < (datetime.datetime.now() - datetime.timedelta(minutes=time_expiration)):
            alarme_to_remove.append(last_warnings[i])

    for i in range(0,len(alarme_to_remove)):
        last_warnings.remove(last_warnings[i])
        
    for i in range(0,len(last_warnings)):
        if last_warnings[i]['operation'] == str(operation) and last_warnings[i]['option_name'] == str(option_name):
            return True
        
    return False
    
    
def add_alarm(operation, option_name):
    warning = dict(operation=str(operation), option_name=str(option_name), timestamp=datetime.datetime.now())
    last_warnings.append(warning)



In [131]:
add_alarm('THL', 'PETRH243_PETRI289')
last_warnings

[{'operation': 'THL',
  'option_name': 'PETRH243_PETRI289',
  'timestamp': datetime.datetime(2021, 7, 16, 13, 33, 29, 966036)}]

In [132]:
check_exists_alarm('THL','PETRH243_PETRI289')

True

In [129]:
last_warnings

[]

KeyError: 'OPTION1OPTION2'